In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import average_precision_score

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from pathlib import Path
import pandas as pd
import torch
import time
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


from model import CNN1D

In [3]:
def generate_fingerprint(smiles: str, radius=2, n_bits=2048):
    """
    Converts a list of SMILES strings to ECFP4 fingerprints.

    Parameters:
        smiles (str): SMILES of a molecule.
        radius (int): Radius parameter for Morgan fingerprint. ECFP4 corresponds to radius=2.
        n_bits (int): Size of the fingerprint bit vector.

    Returns:
        np.ndarray: Array of fingerprint vectors.
    """

    mol = Chem.MolFromSmiles(smiles)

    # See other fingerprint generation methods:
    # https://www.rdkit.org/docs/GettingStartedInPython.html#fingerprinting-and-molecular-similarity
    generator = AllChem.GetMorganGenerator(radius=radius, fpSize=n_bits)
    fingerprint = generator.GetFingerprint(mol)
    return np.array(fingerprint)

In [4]:
# Example Training Loop
def train_model(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=10, device='cuda', weights_path=Path("weights")):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            # Move data to device if using GPU
            inputs, labels = inputs.to(device), labels.to(device)
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_dataloader):.4f}")
        
    gt, pred = validate_model(model, val_dataloader)
    val_score = criterion(torch.from_numpy(pred), torch.from_numpy(gt))
    torch.save(model.state_dict(), weights_path/f"model_{num_epochs}_{val_score}.pt")

    

def validate_model(model, dataloader, device='cuda'):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return y_true, y_pred


In [5]:
def match():
    # TODO Pick the batches by least similar molecules
    # Create a table where for each molecule we have the least similar molecules
    # (e.g. num mcs atoms < (num_mol_atoms*0.3) and num_mcs_bonds < (num_mol_bonds*0.3))
    from rdkit.Chem import rdFMCS
    from rdkit.Chem import Draw

    mol1 = Chem.MolFromSmiles(train['smiles'][0])
    closest_by_atoms_mol = None
    closest_by_bonds_mol = None
    ac = 0
    bc = 0
    for mol in train['smiles'][1:1000]:
        mol2 = Chem.MolFromSmiles(mol)
        mcs = rdFMCS.FindMCS([mol1, mol2])
        mcs_mol = Chem.MolFromSmarts(mcs.smartsString)
        # Draw.MolsToGridImage([mol1, mol2, mcs_mol], molsPerRow=3)
        if mcs.numAtoms > ac:
            ac = mcs.numAtoms
            closest_by_atoms_mol = mcs_mol

        if mcs.numBonds > bc:
            bc = mcs.numBonds
            closest_by_bonds_mol = mcs_mol

    Draw.MolsToGridImage([mol1, closest_by_atoms_mol, closest_by_bonds_mol], molsPerRow=3)

In [6]:
data_root = Path("../data")
weights_path = Path("./weights")
train_file_path = data_root / "train.csv"
test_file_path = data_root / "test.csv"
output_file_path = data_root / "submission.csv"

device = "cuda" if torch.cuda.is_available() else "cpu"

train = pd.read_csv(train_file_path)
test = pd.read_csv(test_file_path)
submission = test.copy()
submission.drop(columns=["smiles"], inplace=True)

In [7]:
batch_size = 32
num_epochs = 200
x_data_file = data_root / "x_data.pt"
y_data_file = data_root / "y_data.pt"
if x_data_file.exists() and y_data_file.exists():
    x = torch.load(x_data_file, weights_only=True)
    y = torch.load(y_data_file, weights_only=True)
else:
    x = torch.stack([torch.from_numpy(generate_fingerprint(s)).float() for s in train["smiles"]]).unsqueeze(1).to(device)
    y = torch.tensor(train["activity"].values).float().to(device)

    torch.save(x, x_data_file)
    torch.save(y, y_data_file)

x_train = x[:int(0.8*len(x))]
y_train = y[:int(0.8*len(y))]

x_val = x[int(0.8*len(x)):]
y_val = y[int(0.8*len(y)):]

x_test = torch.stack([torch.from_numpy(generate_fingerprint(s)).float() for s in test["smiles"]]).unsqueeze(1).to(device)
y_test = torch.zeros(len(x_test), device=device).float()

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
# Model, criterion and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN1D().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.1)

# Train the model
train_model(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=num_epochs, weights_path=weights_path)

ValueError: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1024])) is deprecated. Please ensure they have the same size.

In [9]:
gt, pred = validate_model(model, val_dataloader)
print(criterion(torch.from_numpy(pred), torch.from_numpy(gt)))
pred[pred>=0.5] = 1
pred[pred<0.5] = 0
average_precision_score(gt, pred)

tensor(0.8965)


0.8072797151032477

In [159]:
gt, pred = validate_model(model, test_dataloader)

In [175]:
submission["activity"] = pred

In [177]:
submission.to_csv(output_file_path, index=False)

0          CNC[C@H](c1ccc(Cl)c(Cl)c1)[C@@H](O)c1cccc(N)c1
1       C[C@@H]1Cn2ncc(C3CCN(S(C)(=O)=O)CC3)c2CN1c1ccn...
2       CNC(=O)c1c(NCC2CCC3(CCCC3)CC2)nc(C#N)nc1OCC1CC...
3                 Cc1cc(CNc2nc(N)nc3ccn(Cc4ccccn4)c23)no1
4           C[C@@]1(c2cc(CNCCC(F)(F)F)c(F)cc2F)CCSC(N)=N1
                              ...                        
4211    O=C1COc2ccc(CNC34CCC(CCc5c(F)cnc6ccc(OC[C@@H]7...
4212    CCc1ccc([C@H](CCN2C3CCC2CC(n2c(C)nnc2C(C)C)C3)...
4213          COc1cccc(-c2c(C(=O)N3CCCCC3CO)cc3ccccn23)c1
4214                      Cc1nncn1Cc1cn(-c2ccc(Cl)cc2)nn1
4215    CN(CCCC(=O)NCCO)C(=O)c1ccc2c(c1)c1c(n2C)CC[C@@...
Name: smiles, Length: 4216, dtype: object

,id,smiles,activity
0,0,Cc1ccc2c(N3CCN(CCc4cccc(N5CCNC5=O)c4)CC3)cccc2n1,0.939124
1,1,COc1ccc(Oc2ccc(S(=O)(=O)C3(C(=O)NO)CCC4(CCNCC4...,0.058076
2,2,CN1Cc2ccc(-c3ccc(C[C@@H](C#N)NC(=O)C4(N)CCOCC4...,0.003415
3,3,O=C(NC1CC1c1ccccc1)N1CCC(c2ncon2)CC1,0.360064
4,4,CNc1cc(CCN2CCN(CCc3ccc4c(c3)COC4=O)CC2)ccc1C#N,0.997569
